## policy optimization

In [1]:
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(library(caret)))
suppressWarnings(suppressMessages(library(mlr)))

In [2]:
dir_path = '/export/projects2/jeryang_narrative_project/globe/log'
setwd(dir_path)

### first experiment

In [3]:
# load data from the first experiment 
df1 = read.csv('./df_exp1.csv') # covariates 
df1_s = read.csv('./df_surrogate_exp1.csv') # surrogates

df1 = arrange(df1, subscriber_id)
df1_s = arrange(df1_s, subscriber_id)

In [4]:
optimize_binary_policy <- function(mu, # load predicted outcomes, see details of estimating an outcome model via cross-fitting in cross_fitting.ipynb 
                            df_x, # load data with pre-treatment covariates
                            df_s, # load data with surrogates and outcomes
                            id, # id to match the data with
                            outcome, # specify which outcome to use to optimize the policy 
                            train_id){ # vector of row numbers used in training 
                                   

mu = left_join(mu,select(df_x,all_of(id),p_treated,risk_score,treated), by = id)
mu = left_join(mu,select(df_s,all_of(id),all_of(outcome)), by = id)
mu = rename(mu,rev = all_of(outcome))
mu = arrange(mu, id)

# construct doubly robust scores
mu$y0_dr = ifelse(mu$treated == 1, mu$y0, mu$y0 + (mu$rev - mu$y0)/(1-mu$p_treated))
mu$y1_dr = ifelse(mu$treated == 0, mu$y1, mu$y1 + (mu$rev - mu$y1)/(mu$p_treated))
mu$tau_dr = mu$y1_dr - mu$y0_dr
    
# construct misclassification cost
cost = data.frame(control = rep(0,nrow(mu)), treat = rep(0,nrow(mu)))
cost$control = ifelse(mu$tau_dr < 0,0,abs(mu$tau_dr))
cost$treat = ifelse(mu$tau_dr > 0,0,abs(mu$tau_dr))

# convert covariates into model matrix
df_x = arrange(df_x, id)
df_X = select(df_x,-regi_user_key,-treated,-last_stop_date,
              -churn,-time,-rev,-rev1,-rev2,-rev3,-subscription_status)
df_X = model.matrix(~.-1, df_X) %>% as.data.frame
colnames(df_X) = paste0('col',seq(1,ncol(df_X)))

# specify cost-sensitive classifier
bpo = makeCostSensTask(id = "binary-action policy learning", 
                       data = df_X[train_id,], 
                       cost = cost[train_id,])

# specify learner with pre-tuned hyperparameters 
learner = makeLearner("classif.xgboost",
                      predict.type = "prob",
                      eval_metric = 'logloss',
                      eta = 0.3,
                      max_depth = 6,
                      gamma = 0,
                      colsample_bytree = 0.6,
                      min_child_weight = 1,
                      subsample = 1,
                      nrounds = 150)
learner = makeCostSensWeightedPairsWrapper(learner)

# train the classifier
bpo = mlr::train(learner, bpo)

return(bpo)

}

In [5]:
# convert covariates into model matrix for prediction
df_X = select(df1,-regi_user_key,-treated,-last_stop_date,
              -churn,-time,-rev,-rev1,-rev2,-rev3,-subscription_status)
df_X = model.matrix(~.-1, df_X) %>% as.data.frame
colnames(df_X) = paste0('col',seq(1,ncol(df_X)))

In [6]:
# load predicted outcomes (y0, y1) from outcome models
mu = read.csv('mu_3y.csv') # outcome models trained to predict 3-year revenue

#mu = read.csv('mu_18m_true.csv') # outcome models trained to predict 18-month revenue
#mu = read.csv('mu_18m_6.csv') # outcome models trained to predict surrogate index on 18-month revenue constructed with surrogates in the first 6 months
#mu = read.csv('mu_18m_5.csv') # outcome models trained to predict surrogate index on 18-month revenue constructed with surrogates in the first 5 months
#mu = read.csv('mu_18m_4.csv') # outcome models trained to predict surrogate index on 18-month revenue constructed with surrogates in the first 4 months
#mu = read.csv('mu_18m_3.csv') # outcome models trained to predict surrogate index on 18-month revenue constructed with surrogates in the first 3 months
#mu = read.csv('mu_18m_2.csv') # outcome models trained to predict surrogate index on 18-month revenue constructed with surrogates in the first 2 months
#mu = read.csv('mu_18m_1.csv') # outcome models trained to predict surrogate index on 18-month revenue constructed with surrogates in the first 1 month

#mu = read.csv('mu_6m.csv') # outcome models trained to predict 6-month revenue
#mu = read.csv('mu_5m.csv') # outcome models trained to predict 5-month revenue
#mu = read.csv('mu_4m.csv') # outcome models trained to predict 4-month revenue
#mu = read.csv('mu_3m.csv') # outcome models trained to predict 3-month revenue
#mu = read.csv('mu_2m.csv') # outcome models trained to predict 2-month revenue
#mu = read.csv('mu_1m.csv') # outcome models trained to predict 1-month revenue

mu = arrange(mu, subscriber_id)

In [7]:
# split training and test data, test data will be used in policy evaluation
set.seed(101)
train_id = sort(sample(nrow(df_X), nrow(df_X)*0.8))

In [8]:
# optimize policy
obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev3', train_id) # optimize policy with 3-year surrogate index

#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev', train_id) # optimize policy with 18-month revenue
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_6', train_id) # optimize policy with 18-month surrogate index constructed with surrogates in the first 6 months
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_5', train_id) # optimize policy with 3-year surrogate index constructed with surrogates in the first 5 months
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_4', train_id) # optimize policy with 3-year surrogate index constructed with surrogates in the first 4 months
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_3', train_id) # optimize policy with 3-year surrogate index constructed with surrogates in the first 3 months
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_2', train_id) # optimize policy with 3-year surrogate index constructed with surrogates in the first 2 months
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_1', train_id) # optimize policy with 3-year surrogate index constructed with surrogates in the first 1 month

#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_m6', train_id) # optimize policy with 6-month revenue
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_m5', train_id) # optimize policy with 5-month revenue
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_m4', train_id) # optimize policy with 4-month revenue
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_m3', train_id) # optimize policy with 3-month revenue
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_m2', train_id) # optimize policy with 2-month revenue
#obp = optimize_binary_policy(mu, df1, df1_s, 'subscriber_id', 'rev_m1', train_id) # optimize policy with 1-month revenue

In [9]:
# show classification on test data
pred = predict(obp, newdata = df_X[-train_id,]) %>% as.data.frame
table(pred$response)


control   treat 
   3194    5164 

### second experiment

In [10]:
# load data from the second experiment 
df2 = read.csv('./df_exp2.csv')
df2_s = read.csv('./df_surrogate_exp2.csv')

df2 = arrange(df2, subscriber_id)
df2_s = arrange(df2_s, subscriber_id)

In [11]:
mu2 = read.csv('mu2_3y.csv')
mu2 = left_join(mu2, select(df2_s, subscriber_id, rev3), by = 'subscriber_id')
colnames(mu2)[9] = 'rev'

mu2 = left_join(mu2, select(df2,subscriber_id,condition,treated,prob), by = 'subscriber_id')

mu2$y0_dr = with(mu2, ifelse(condition == 'control', y0 + (rev-y0)/(1-prob), y0))
mu2$y1_dr = with(mu2, ifelse(condition == 'gift card', y1 + (rev-y1)/(prob/6), y1))
mu2$y2_dr = with(mu2, ifelse(condition == 'thank you email only', y2 + (rev-y2)/(prob/6), y2))
mu2$y3_dr = with(mu2, ifelse(condition == '$3.99/8 weeks', y3 + (rev-y3)/(prob/6), y3))
mu2$y4_dr = with(mu2, ifelse(condition == '$4.99/8 weeks', y4 + (rev-y4)/(prob/6), y4))
mu2$y5_dr = with(mu2, ifelse(condition == '$5.99/8 weeks', y5 + (rev-y5)/(prob/6), y5))
mu2$y6_dr = with(mu2, ifelse(condition == '$5.99/4 weeks', y6 + (rev-y6)/(prob/6), y6))

mu2 = arrange(mu2, subscriber_id)

In [12]:
cost2 = mu2[,13:19]
cost2$max = apply(cost2,1,max)
cost2 = apply(cost2,2,function(x) abs(x - cost2$max)) %>% as.data.frame
cost2$max = NULL

In [13]:
head(cost2)             

,y0_dr,y1_dr,y2_dr,y3_dr,y4_dr,y5_dr,y6_dr
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,211.5463,572.4547,0.00000,601.0201,623.8939,198.61285,253.0677
2,146.9413,426.9260,152.03772,379.9951,243.2512,50.62915,0.0000
3,20.8282,260.2997,44.54987,430.0086,185.5098,13.97638,0.0000
4,0.0000,564.3921,183.05039,678.5666,162.1289,216.32974,303.7049
5,611.4573,819.0956,762.88391,956.6161,703.2367,0.00000,487.7246
6,0.0000,213.3935,473.72961,745.4255,460.3596,533.43066,351.7082


In [14]:
df2_x = select(df2, -c('subscriber_id',
                 'treated', 'churn', 'time', 'condition', 'prob', 'hte', 'action', 'score_c', 'rev1', 
                 'rev','subscription_status','score_discrete','last_stop_date'),
                  -contains('std'))
df2_X = model.matrix(~.-1, df2_x)
df2_X = as.data.frame(df2_X)
colnames(df2_X) = paste0('col',seq(1,ncol(df2_X)))

In [15]:
set.seed(101)
train2_id = sort(sample(nrow(df2_X), nrow(df2_X)*0.8))

In [16]:
omp = makeCostSensTask(id = "optimize multi-action policy", 
                       data = df2_X[train2_id,], 
                       cost = cost2[train2_id,])

learner = makeLearner("classif.xgboost",
                      predict.type = "prob",
                      eval_metric = 'logloss',
                      eta = 0.3,
                      max_depth = 6,
                      gamma = 0,
                      colsample_bytree = 0.6,
                      min_child_weight = 1,
                      subsample = 1,
                      nrounds = 150)
learner = makeCostSensWeightedPairsWrapper(learner)
    
omp = mlr::train(learner, omp)
pred = predict(omp, newdata = df2_X[-train_id,]) %>% as.data.frame

In [17]:
pred = pred %>% 
mutate(response = case_when(response == 'y0_dr' ~ 'control',
                      response == 'y1_dr' ~ 'gift card',
                      response == 'y2_dr' ~ 'thank you email only',
                      response == 'y3_dr' ~ '$3.99/8 weeks',
                      response == 'y4_dr' ~ '$4.99/8 weeks',
                      response == 'y5_dr' ~ '$5.99/8 weeks',
                      response == 'y6_dr' ~ '$5.99/4 weeks',
                     ))

In [18]:
# table 1: show classification on test data
table(pred$response)


       $3.99/8 weeks        $4.99/8 weeks        $5.99/4 weeks 
                 122                  208                17821 
       $5.99/8 weeks              control            gift card 
               16464                11736                  167 
thank you email only 
               15608 

In [19]:
table(pred$response)/nrow(pred)


       $3.99/8 weeks        $4.99/8 weeks        $5.99/4 weeks 
         0.001963751          0.003348035          0.286852526 
       $5.99/8 weeks              control            gift card 
         0.265009819          0.188906416          0.002688086 
thank you email only 
         0.251231369 